In [1]:
import numpy as np
import pandas as pd
import math
import pymatgen as mg
from pymatgen.transformations.standard_transformations import RemoveSpeciesTransformation
from pymatgen.transformations.standard_transformations import SupercellTransformation

In [22]:
structure = mg.Structure.from_file("cif/1D/1555295.cif")

cell = [structure.lattice.a, structure.lattice.b, structure.lattice.c]
print(cell)

u = SupercellTransformation(scaling_matrix=((2, 0, 0), (0, 2, 0), (0, 0, 2)))
supercell = u.apply_transformation(structure)

t = RemoveSpeciesTransformation(["H", "C", "N", "S", "O", "I"])
modified_structure = t.apply_transformation(supercell)

print(len(modified_structure))

# Pb-Pb distances in a table

dumb = np.array([modified_structure.get_distance(0, i) for i in range(1, len(modified_structure))])
# around
dumb1 = np.around(dumb,decimals=2)
# remove duplicates
dumb2 = np.array(list(set(dumb1)))
# sort values
dist_lst = np.sort(dumb2)
print(len(dist_lst))
print(dist_lst)

[8.0816, 12.390599999999997, 12.9013]
16
11
[ 4.04  8.08 12.06 12.38 12.39 12.9  13.04 13.15 15.29 15.45 16.62]


In [23]:
# Find d_min = shortest Pb-Pb bond
# Usually around 4.06 in 1D
d_min = min(dist_lst)
print(d_min)

#for i in range(0, len(dist_lst)-1):
#    a = round(dist_lst[i]**2, 2)
#    b = round(dist_lst[i+1]**2 - d_min**2, 2)
#    #print(a, b)
#    print(math.isclose(a, b, abs_tol = 0.2))

# Remove duplicate distances due to short Pb-Pb distances    
def duplicates(dist_lst):
    for i in range(0, len(dist_lst)-1):
        a = round(dist_lst[i]**2, 2)
        b = round(dist_lst[i+1]**2 - d_min**2, 2)
        if math.isclose(a, b, abs_tol = 0.2):
            dist_lst[i+1] = dist_lst[i]
    return dist_lst

4.04


In [24]:
dst_lst_new = duplicates(dist_lst)
print(dst_lst_new)

[ 4.04  8.08 12.06 12.38 12.39 12.9  13.04 13.15 15.29 15.45 16.62]


In [28]:
# remove duplicates
dumb2 = np.array(list(set(dst_lst_new)))

# Crop shortest Pb-Pb and Pb-Pb-Pb bond
dist_lst_min = np.delete(dumb2, np.argwhere((dumb2 <= 2*d_min+0.01)))
print('min', dist_lst_min)

d_max = max(cell)
print(d_max)

# Crop higher distances list
dist_lst_crop = np.delete(dist_lst_min, np.argwhere((dist_lst_min >= d_max+0.01)))
print(dist_lst_crop)

min [12.39 12.06 12.38 12.9  13.04 13.15 15.29 15.45 16.62]
12.9013
[12.39 12.06 12.38 12.9 ]


In [29]:
# Problem with either the edge or diagonal of the losange
# One diagonal is half longer axis
a = dist_lst_crop[0]
b = dist_lst_crop[1]
print(a, b)

c = max(dist_lst_crop)
print(c)

c_test1 = b**2 - (a/2)**2
c_test2 = a**2 - (b/2)**2
print(c_test1, c_test2, (c/2)**2)

if math.isclose(c_test1, (c/2)**2, abs_tol = 0.2):
    diag1 = a
    diag2 = c
    
if math.isclose(c_test2, (c/2)**2, abs_tol = 0.2):
    diag1 = b
    diag2 = c

12.39 12.06
12.9
107.065575 117.1512 41.6025


In [30]:
#Surface defined by losange
print(diag1, diag2)
surf = round(diag1*diag2/2 , 2)
print(surf, "A2")

NameError: name 'diag1' is not defined